In [1]:
import pandas as pd

In [2]:
#import sentiments 
filename='market_sentiment_score (4)'
df=pd.read_csv(filename+'.csv', parse_dates=['timestamp'])
df

,total_score,keyword_score,user_score,timestamp
0,0.050725,0.037594,1.0,2021-06-01 00:01:00
1,0.052980,0.075342,-1.0,2021-06-01 00:02:00
2,0.190184,0.187500,0.0,2021-06-01 00:03:00
3,0.012903,0.013072,0.0,2021-06-01 00:04:00
4,0.032051,0.032468,0.0,2021-06-01 00:05:00
...,...,...,...,...
69349,-0.123077,-0.123077,0.0,2021-07-19 07:16:00
69350,-0.018349,-0.019048,-1.0,2021-07-19 07:17:00
69351,-0.191489,-0.181159,0.0,2021-07-19 07:18:00
69352,-0.115385,-0.120000,0.0,2021-07-19 07:19:00


In [4]:
df.rename(columns={'timestamp':'time'}, inplace=True)
series = pd.Series(df['total_score'].values.tolist(), index=df['time'])
series

time
2021-06-01 00:01:00    0.050725
2021-06-01 00:02:00    0.052980
2021-06-01 00:03:00    0.190184
2021-06-01 00:04:00    0.012903
2021-06-01 00:05:00    0.032051
                         ...   
2021-07-19 07:16:00   -0.123077
2021-07-19 07:17:00   -0.018349
2021-07-19 07:18:00   -0.191489
2021-07-19 07:19:00   -0.115385
2021-07-19 07:20:00   -0.018519
Length: 69354, dtype: float64

In [8]:
#take mean of total_sentiments over 3 hours

hourSeries = series.resample('180T').mean() 
hourDF = hourSeries.to_frame()
hourDF.rename(columns={0:'sentiment_score'}, inplace=True)
hourDF=hourDF.reset_index()
hourDF

,time,sentiment_score
0,2021-06-01 00:00:00,0.032345
1,2021-06-01 03:00:00,0.039159
2,2021-06-01 06:00:00,0.020239
3,2021-06-01 09:00:00,0.006470
4,2021-06-01 12:00:00,0.085424
...,...,...
382,2021-07-18 18:00:00,-0.013505
383,2021-07-18 21:00:00,-0.054870
384,2021-07-19 00:00:00,-0.046855
385,2021-07-19 03:00:00,0.005218


In [9]:
#import tweet volumes
filename='market_tweets_raw_volumes (1)'
df2=pd.read_csv(filename+'.csv', parse_dates=['timestamp'])
df2

,timestamp,count
0,2021-06-01 00:00:00,241
1,2021-06-01 00:01:00,138
2,2021-06-01 00:02:00,151
3,2021-06-01 00:03:00,163
4,2021-06-01 00:04:00,155
...,...,...
69555,2021-07-19 07:15:00,140
69556,2021-07-19 07:16:00,130
69557,2021-07-19 07:17:00,109
69558,2021-07-19 07:18:00,141


In [18]:

df2.rename(columns={'timestamp':'time'}, inplace=True)
series2 = pd.Series(df2['count'].values.tolist(), index=df2['time'])
series2

time
2021-06-01 00:00:00    241
2021-06-01 00:01:00    138
2021-06-01 00:02:00    151
2021-06-01 00:03:00    163
2021-06-01 00:04:00    155
                      ... 
2021-07-19 07:15:00    140
2021-07-19 07:16:00    130
2021-07-19 07:17:00    109
2021-07-19 07:18:00    141
2021-07-19 07:19:00     94
Length: 69560, dtype: int64

In [19]:
#take sum of all tweets over 3 hours
hourSeries2 = series2.resample('180T').sum()

In [20]:
hourDF2 = hourSeries2.to_frame()
hourDF2.rename(columns={0:'tweet volume'}, inplace=True)
hourDF2=hourDF2.reset_index()
hourDF2

,time,tweet volume
0,2021-06-01 00:00:00,23568
1,2021-06-01 03:00:00,19750
2,2021-06-01 06:00:00,23477
3,2021-06-01 09:00:00,24964
4,2021-06-01 12:00:00,36552
...,...,...
382,2021-07-18 18:00:00,25701
383,2021-07-18 21:00:00,19954
384,2021-07-19 00:00:00,17920
385,2021-07-19 03:00:00,18067


In [21]:
mergedDF = pd.merge(hourDF, hourDF2, on ='time', how ='inner')
mergedDF

,time,sentiment_score,tweet volume
0,2021-06-01 00:00:00,0.032345,23568
1,2021-06-01 03:00:00,0.039159,19750
2,2021-06-01 06:00:00,0.020239,23477
3,2021-06-01 09:00:00,0.006470,24964
4,2021-06-01 12:00:00,0.085424,36552
...,...,...,...
382,2021-07-18 18:00:00,-0.013505,25701
383,2021-07-18 21:00:00,-0.054870,19954
384,2021-07-19 00:00:00,-0.046855,17920
385,2021-07-19 03:00:00,0.005218,18067


In [22]:
#multiply mean of sentiment score with total tweet volume
mergedDF['Weighted Sentiment']= mergedDF['sentiment_score']*mergedDF['tweet volume']
mergedDF

,time,sentiment_score,tweet volume,Weighted Sentiment
0,2021-06-01 00:00:00,0.032345,23568,762.299381
1,2021-06-01 03:00:00,0.039159,19750,773.391426
2,2021-06-01 06:00:00,0.020239,23477,475.157496
3,2021-06-01 09:00:00,0.006470,24964,161.527817
4,2021-06-01 12:00:00,0.085424,36552,3122.424180
...,...,...,...,...
382,2021-07-18 18:00:00,-0.013505,25701,-347.083225
383,2021-07-18 21:00:00,-0.054870,19954,-1094.873519
384,2021-07-19 00:00:00,-0.046855,17920,-839.644715
385,2021-07-19 03:00:00,0.005218,18067,94.279057


In [24]:
mergedDF.to_csv('SentimentData_extension.csv', index=False)

In [102]:
filename='ZRX'
asset_df=pd.read_csv('ohlc60m/'+filename+'_cbpro_ohlc_60m.csv', parse_dates=['time'], date_parser=lambda epoch: pd.to_datetime(epoch, unit='s'))
asset_df

,time,low,high,open,close,volume
0,2019-01-01 00:00:00,0.289651,0.294562,0.292668,0.293986,231775.61292
1,2019-01-01 01:00:00,0.292059,0.296000,0.293981,0.294199,184073.50054
2,2019-01-01 02:00:00,0.290267,0.294799,0.294163,0.293420,79092.22161
3,2019-01-01 03:00:00,0.291263,0.297987,0.291263,0.297239,145928.41373
4,2019-01-01 04:00:00,0.295312,0.300000,0.297229,0.296052,159515.01337
...,...,...,...,...,...,...
22285,2021-07-17 13:00:00,0.611797,0.620958,0.618030,0.616280,128858.72679
22286,2021-07-17 14:00:00,0.614776,0.618920,0.615160,0.615870,59223.25132
22287,2021-07-17 15:00:00,0.608063,0.615174,0.614831,0.614791,114849.95673
22288,2021-07-17 16:00:00,0.611196,0.623232,0.614618,0.618063,85856.80589


In [103]:
series_asset = pd.Series(asset_df['open'].values.tolist(), index=asset_df['time'])

series_asset_vol = pd.Series(asset_df['volume'].values.tolist(), index=asset_df['time'])

In [104]:
#Take mean of open values
asset_hourSeries = series_asset.resample('180T').mean()

#take sum of volume
asset_hourSeries_vol = series_asset_vol.resample('180T').sum()

print(asset_hourSeries)
asset_hourSeries_vol

time
2019-01-01 00:00:00    0.293604
2019-01-01 03:00:00    0.295503
2019-01-01 06:00:00    0.299259
2019-01-01 09:00:00    0.300422
2019-01-01 12:00:00    0.301787
                         ...   
2021-07-17 03:00:00    0.607872
2021-07-17 06:00:00    0.618737
2021-07-17 09:00:00    0.616854
2021-07-17 12:00:00    0.614843
2021-07-17 15:00:00    0.616051
Freq: 180T, Length: 7430, dtype: float64


time
2019-01-01 00:00:00    494941.33507
2019-01-01 03:00:00    350331.95011
2019-01-01 06:00:00    142890.20891
2019-01-01 09:00:00    112887.35553
2019-01-01 12:00:00    123500.72430
                           ...     
2021-07-17 03:00:00    431836.59918
2021-07-17 06:00:00    267068.62457
2021-07-17 09:00:00    283922.70331
2021-07-17 12:00:00    213250.53189
2021-07-17 15:00:00    231214.36632
Freq: 180T, Length: 7430, dtype: float64

In [105]:
asset_hourDF = asset_hourSeries.to_frame()
asset_hourDF.rename(columns={0:'open '+filename}, inplace=True)

asset_vol_hourDF = asset_hourSeries_vol.to_frame()
asset_vol_hourDF.rename(columns={0:'Volume '+filename}, inplace=True)


mergedHourDF = pd.concat([asset_vol_hourDF, asset_hourDF], axis=1)
mergedHourDF

,Volume ZRX,open ZRX
time,,
2019-01-01 00:00:00,494941.33507,0.293604
2019-01-01 03:00:00,350331.95011,0.295503
2019-01-01 06:00:00,142890.20891,0.299259
2019-01-01 09:00:00,112887.35553,0.300422
2019-01-01 12:00:00,123500.72430,0.301787
...,...,...
2021-07-17 03:00:00,431836.59918,0.607872
2021-07-17 06:00:00,267068.62457,0.618737
2021-07-17 09:00:00,283922.70331,0.616854


In [106]:
finalDF = pd.merge(mergedDF, mergedHourDF, on ='time', how ='inner')
finalDF

,time,sentiment_score,tweet volume,Weighted Sentiment,Volume ZRX,open ZRX
0,2021-06-01 00:00:00,0.032345,23568,762.299381,925771.77054,1.013978
1,2021-06-01 03:00:00,0.039159,19750,773.391426,770861.84754,1.012304
2,2021-06-01 06:00:00,0.020239,23477,475.157496,675310.71259,1.039954
3,2021-06-01 09:00:00,0.006470,24964,161.527817,732448.58717,1.009778
4,2021-06-01 12:00:00,0.085424,36552,3122.424180,807479.30396,1.025217
...,...,...,...,...,...,...
369,2021-07-17 03:00:00,-0.010676,18590,-198.462543,431836.59918,0.607872
370,2021-07-17 06:00:00,-0.016053,19695,-316.161844,267068.62457,0.618737
371,2021-07-17 09:00:00,-0.007262,20159,-146.389514,283922.70331,0.616854
372,2021-07-17 12:00:00,-0.041026,26502,-1087.262508,213250.53189,0.614843


In [107]:
finalDF.to_csv('extendedDatasets/'+filename+'.csv', index= False)